In [45]:
!pip install tabula-py
!pip install PyPDF2
import os
import json
import re
import unicodedata
import PyPDF2
import tabula
import unicodedata

def normalize_string(s):
    # Split the string into words
    words = s.split()
    
    # List of words to keep lowercase
    lowercase_words = ['de', 'del', 'la', 'las', 'los', 'el']
    
    # Capitalize each word unless it's in the lowercase_words list
    # Always capitalize the first word
    normalized_words = []
    for i, word in enumerate(words):
        if i == 0 or word.lower() not in lowercase_words:
            normalized_words.append(word.capitalize())
        else:
            normalized_words.append(word.lower())
    
    # Join the words back together
    return ' '.join(normalized_words)

def process_data(input_file, output_file):
    series_dict = {}

    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(maxsplit=1)
            if len(parts) == 2:
                series, locality = parts
                series_code = series[:3]
                normalized_locality = normalize_string(locality)
                
                if series_code not in series_dict:
                    series_dict[series_code] = normalized_locality

    sorted_dict = dict(sorted(series_dict.items()))

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(sorted_dict, f, ensure_ascii=False, indent=2)

    print(f"JSON dictionary has been saved to {output_file}")


import re

def extract_data_from_pdf(pdf_path):
    data = []
    # Read all pages
    tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)
    
    for table in tables:
        if len(table.columns) >= 3:  # Ensure we have at least 3 columns
            for _, row in table.iterrows():
                series = row.iloc[1][:3]  # Get first 3 characters of the second column
                locality = row.iloc[2]  # Get the third column (locality)
                if isinstance(series, str) and isinstance(locality, str):
                    data.append((series, locality))
    
    return data


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
def process_department(department):
    department_lower = department.lower()
    base_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
    pdf_path = os.path.join(base_dir, "maps", "planescircuitales", f"plan_{department_lower}.pdf")
    output_path = os.path.join(base_dir, "maps", "zonasxseries", f"mapping_{department_lower}.json")

    print(f"PDF path: {pdf_path}")
    print(f"Output path: {output_path}")

    if not os.path.exists(pdf_path):
        print(f"Error: PDF file not found at {pdf_path}")
        return

    # Extract and process data directly from PDF
    extracted_data = extract_data_from_pdf(pdf_path)
    print(f"Extracted data: {extracted_data[:5]}...")  # Print first 5 items
    print(f"Total extracted items: {len(extracted_data)}")

 # Process the extracted data
    series_dict = {}
    for series, locality in extracted_data:
        if series not in series_dict:
            series_dict[series] = normalize_string(locality)

    sorted_dict = dict(sorted(series_dict.items()))

    print(f"Processed data: {list(sorted_dict.items())[:5]}...")  # Print first 5 items
    print(f"Total unique series: {len(sorted_dict)}")

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(sorted_dict, f, ensure_ascii=False, indent=2)

    print(f"Processing completed for {department}")

In [47]:
# Execute this cell with the desired department name
department = "Canelones"  # Change this to the department you want to process
process_department(department)

PDF path: c:\Users\trico\Desktop\Internas 2024\maps\planescircuitales\plan_canelones.pdf
Output path: c:\Users\trico\Desktop\Internas 2024\maps\zonasxseries\mapping_canelones.json
Extracted data: [('CAA', 'CANELONES'), ('CAA', 'CANELONES'), ('CAA', 'CANELONES'), ('CAA', 'CANELONES'), ('CAA', 'CANELONES')]...
Total extracted items: 1041
Processed data: [('CAA', 'Canelones'), ('CBA', 'Santa Lucía'), ('CBB', 'Santa Lucía'), ('CBC', 'Santa Lucía'), ('CCA', 'Los Cerrillos')]...
Total unique series: 43
Processing completed for Canelones
